<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/issue/c620_real_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import joblib
!pip install autorch > log.txt
c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')

In [2]:
c.keys()

dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])

# 真實數據 wt數據

In [3]:
path = '/content/drive/MyDrive/台塑輕油案子/data/c620/明志_遷移式學習_訓練資料_寄送版/蒸餾塔(C620_C660_C670)取出品管資料_寄送明志科大 r2.xlsx'
df_real = pd.read_excel(path,sheet_name='資料彙整(寄送明志)r2')
df_real.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Unnamed: 146,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Unnamed: 151,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169
0,NaN,C620_Vent_Gas,2020-09-02 00:00:00,2020-09-03 00:00:00,2020-09-04 00:00:00,2020-09-05 00:00:00,2020-09-06 00:00:00,2020-09-07 00:00:00,2020-09-08 00:00:00,2020-09-09 00:00:00,2020-09-10 00:00:00,2020-09-11 00:00:00,2020-09-12 00:00:00,2020-09-13 00:00:00,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,2020-09-20 00:00:00,2020-09-21 00:00:00,2020-09-22 00:00:00,2020-09-23 00:00:00,2020-09-24 00:00:00,2020-09-25 00:00:00,2020-09-26 00:00:00,2020-09-27 00:00:00,2020-09-28 00:00:00,2020-09-29 00:00:00,2020-09-30 00:00:00,2020-10-02 00:00:00,2020-10-03 00:00:00,2020-10-04 00:00:00,2020-10-05 00:00:00,2020-10-06 00:00:00,2020-10-07 00:00:00,2020-10-08 00:00:00,2020-10-09 00:00:00,2020-10-10 00:00:00,...,2021-01-12 00:00:00,2021-01-13 00:00:00,2021-01-14 00:00:00,2021-01-15 00:00:00,2021-01-16 00:00:00,2021-01-17 00:00:00,2021-01-18 00:00:00,2021-01-19 00:00:00,2021-01-20 00:00:00,2021-01-21 00:00:00,2021-01-22 00:00:00,2021-01-23 00:00:00,2021-01-24 00:00:00,2021-01-25 00:00:00,2021-01-26 00:00:00,2021-01-27 00:00:00,2021-01-28 00:00:00,2021-01-29 00:00:00,2021-01-30 00:00:00,2021-01-31 00:00:00,2021-02-02 00:00:00,2021-02-03 00:00:00,2021-02-04 00:00:00,2021-02-05 00:00:00,2021-02-06 00:00:00,2021-02-07 00:00:00,2021-02-08 00:00:00,2021-02-09 00:00:00,2021-02-10 00:00:00,2021-02-11 00:00:00,2021-02-12 00:00:00,2021-02-13 00:00:00,2021-02-14 00:00:00,2021-02-15 00:00:00,2021-02-16 00:00:00,2021-02-17 00:00:00,2021-02-18 00:00:00,2021-02-19 00:00:00,2021-02-20 00:00:00,2021-02-21 00:00:00
1,NaN,Hydrogen,6.531,8.41,7.918,7.8752,7.372,7.59,7.431,7.115,7.37,6.78,7.96,7.79,8.21,8.1288,7.6925,8.1227,8.8135,7.6956,8.041,7.907,7.394,7.774,8.159,7.22,7.184,6.901,7.151,7.39,7.18,8.86,8.1,8.5,8.0328,11.008,8.0358,10.241,30.628,36.404,...,9.56,8.6134,8.5596,4.7681,9.9246,8.5458,8.902,6.704,8.496,8.01,8.815,7.868,7.988,8.315,8.65,7.809,7.81,9.12,8.72,9.45,9.6649,9.0969,8.6988,3.6305,9.3991,9.107,8.271,7.902,8.084,7.941,7.491,8.036,7.967,7.65,7.99,7.63,8.47,9.25,9.86,9.61
2,NaN,Methane,1.898,2.229,2.343,2.453,2.384,2.414,2.371,2.316,2.36,2.31,2.57,2.41,2.53,2.5528,2.4797,2.5428,2.7001,2.3819,2.399,2.298,2.315,2.163,2.212,2.322,2.296,2.21,2.256,2.233,2.22,2.48,2.45,2.51,2.3331,2.8984,2.2278,2.3713,3.0879,3.642,...,3,2.8436,2.7739,1.4418,2.4487,2.8594,2.776,2.384,2.798,2.711,2.964,2.742,2.694,2.824,3.031,3.173,3.17,3.21,3.09,3.22,3.3915,3.1516,3.2169,1.6245,3.4361,3.168,2.982,2.973,2.901,2.872,2.701,2.911,2.934,2.93,2.926,2.83,2.99,3.27,2.83,3.46
3,NaN,Ethane,35.025,38.57,37.649,37.257,38.169,37.37,37.082,37.207,37.17,37.19,38.94,38.94,39.6,39.391,39.359,40.237,39.285,36.894,37.034,36.761,37.409,36.568,36.671,37.315,36.726,37.814,37.294,37.049,38.11,34.88,35.12,34.83,34.477,38.151,31.911,35.016,28.548,30.702,...,38.32,36.366,32.599,25.409,34.855,35.881,34.237,31.999,32.443,33.041,34.721,32.939,33.654,34.075,35.408,37.122,37.12,35.29,35.12,34.86,33.974,32.74,33.978,29.971,34.754,33.463,32.24,32.04,

In [4]:
c620_wt1,c620_fout1 = df_real.iloc[1:1+41,2:].T,df_real.iloc[[1+41],2:].T/1000 
c620_wt2,c620_fout2 = df_real.iloc[45:45+41,2:].T,df_real.iloc[[45+41],2:].T/1000  
c620_wt3,c620_fout3 = df_real.iloc[89:89+41,2:].T,df_real.iloc[[89+41],2:].T/1000 
c620_wt4,c620_fout4 = df_real.iloc[133:133+41,2:].T,df_real.iloc[[133+41],2:].T/1000
c620_feed_wt,c620_feed_flow = df_real.iloc[441:441+41,2:].T,df_real.iloc[[441+41],2:].T/1000
idx = df_real.iloc[0,2:].values

c620_wt1.index = idx
c620_wt2.index = idx
c620_wt3.index = idx
c620_wt4.index = idx

c620_wt1.columns = c['vent_gas_x']
c620_wt2.columns = c['distillate_x']
c620_wt3.columns = c['sidedraw_x']
c620_wt4.columns = c['bottoms_x']

In [5]:
z1 = (c620_wt1*c620_fout1.values.reshape(-1,1)).sum(axis=1)
z2 = (c620_wt2*c620_fout2.values.reshape(-1,1)).sum(axis=1)
z3 = (c620_wt3*c620_fout3.values.reshape(-1,1)).sum(axis=1)
z4 = (c620_wt4*c620_fout4.values.reshape(-1,1)).sum(axis=1)
z5 = (c620_feed_wt*c620_feed_flow.values.reshape(-1,1)).sum(axis=1)
z5.index = idx
print(z1+z2+z3+z4)
print('===========')
print(z5)

2020-09-02    100427.247487
2020-09-03     95708.918604
2020-09-04     93182.356770
2020-09-05     96117.822961
2020-09-06     96911.239283
                  ...      
2021-02-17    100744.988582
2021-02-18    101797.675365
2021-02-19     86208.095809
2021-02-20     92387.407846
2021-02-21     92612.214007
Length: 168, dtype: float64
2020-09-02    16020.738198
2020-09-03    15956.411496
2020-09-04    16185.885944
2020-09-05    16672.607007
2020-09-06    16625.489224
                  ...     
2021-02-17    17148.395591
2021-02-18    17279.930723
2021-02-19    16697.473962
2021-02-20    16176.879703
2021-02-21    17790.634499
Length: 168, dtype: float64


# 模擬數據

In [6]:
df_sim = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0)
df_sim_mb = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_mbcheck.csv',index_col=0)

In [7]:
c620_wt1,c620_f1 = df_sim[c['vent_gas_x']],df_sim_mb['Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Mass Flow Rate_ton/hr']
c620_wt2,c620_f2 = df_sim[c['distillate_x']],df_sim_mb['Tatoray Stripper C620 Operation_Distillate Production Rate and Composition_Mass Flow Rate_ton/hr']
c620_wt3,c620_f3 = df_sim[c['sidedraw_x']],df_sim_mb['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Mass Flow Rate_ton/hr']
c620_wt4,c620_f4 = df_sim[c['bottoms_x']],df_sim_mb['Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Mass Flow Rate_ton/hr']
c620_feed_wt,c620_feed_flow = df_sim[c['x41']],df_sim_mb['Combined Feed to Tatoray Stripper C620_Flow Rate_Mass Flow Rate_ton/hr']

In [8]:
z1 = (c620_wt1*c620_f1.values.reshape(-1,1)).sum(axis=1)
z2 = (c620_wt2*c620_f2.values.reshape(-1,1)).sum(axis=1)
z3 = (c620_wt3*c620_f3.values.reshape(-1,1)).sum(axis=1)
z4 = (c620_wt4*c620_f4.values.reshape(-1,1)).sum(axis=1)
z5 = (c620_feed_wt*c620_feed_flow.values.reshape(-1,1)).sum(axis=1)
print(z1+z2+z3+z4)
print('===========')
print(z5)

001-001    15348.305227
001-002    15348.306044
001-003    15348.306283
001-004    15348.304367
001-005    15348.304304
               ...     
144-104    18587.587976
144-105    18587.587189
144-106    18587.582646
144-107    18587.582646
144-108    18587.582646
Length: 15552, dtype: float64
001-001    15348.305846
001-002    15348.305846
001-003    15348.305846
001-004    15348.305846
001-005    15348.305846
               ...     
144-104    18587.580181
144-105    18587.580181
144-106    18587.580181
144-107    18587.580181
144-108    18587.580181
Length: 15552, dtype: float64
